In [3]:
import numpy as np
from astropy.io import fits
from astropy.table import Table

import sys
sys.path.append('/Users/nityaravi/Documents/Github/RotationCurves/spirals/')
from dark_matter_mass_v1 import rot_fit_BB

In [5]:
OUTPUT_FOLDER = '/Users/nityaravi/Documents/Research/RotationCurves/data/manga/output_files/DR17/'
HESS_FOLDER = OUTPUT_FOLDER + 'DRP_vel_map_Hessians/Hessians/'


master_table_fn = OUTPUT_FOLDER + '/H_alpha_HIvel_BB_extinction_H2_R90.fits'

master_table = Table.read(master_table_fn, format='fits')

In [7]:
master_table[0:5]

plate,ifudsgn,plateifu,mangaid,versdrp2,versdrp3,verscore,versutil,versprim,platetyp,srvymode,objra,objdec,ifuglon,ifuglat,ifura,ifudec,ebvgal,nexp,exptime,drp3qual,bluesn2,redsn2,harname,frlplug,cartid,designid,cenra,cendec,airmsmin,airmsmed,airmsmax,seemin,seemed,seemax,transmin,transmed,transmax,mjdmin,mjdmed,mjdmax,gfwhm,rfwhm,ifwhm,zfwhm,mngtarg1,mngtarg2,mngtarg3,catidnum,plttarg,manga_tileid,nsa_iauname,ifudesignsize,ifutargetsize,ifudesignwrongsize,z,zmin,zmax,szmin,szmax,ezmin,ezmax,probs,pweight,psweight,psrweight,sweight,srweight,eweight,esweight,esrweight,nsa_field,nsa_run,nsa_camcol,nsa_version,nsa_nsaid,nsa_nsaid_v1b,nsa_z,nsa_zdist,nsa_sersic_absmag [7],nsa_elpetro_absmag [7],nsa_elpetro_amivar [7],nsa_sersic_mass,nsa_elpetro_mass,nsa_elpetro_ba,nsa_elpetro_phi,nsa_extinction [7],nsa_elpetro_th50_r,nsa_petro_th50,nsa_petro_flux [7],nsa_petro_flux_ivar [7],nsa_elpetro_flux [7],nsa_elpetro_flux_ivar [7],nsa_sersic_ba,nsa_sersic_n,nsa_sersic_phi,nsa_sersic_th50,nsa_sersic_flux [7],nsa_sersic_flux_ivar [7],smoothness_score,nsa_elpetro_th90,v_sys,v_sys_err,ba,ba_err,x0,x0_err,y0,y0_err,phi,phi_err,v_max,v_max_err,r_turn,r_turn_err,chi2,alpha,alpha_err,Rmax,M,M_err,fit_flag,Sigma_disk,Sigma_disk_err,R_disk,R_disk_err,rho_bulge,rho_bulge_err,R_bulge,R_bulge_err,M90_disk,M90_disk_err,M_disk,M_disk_err,chi2_disk,logHI,logHIlim,WF50,WF50_err,DL_ttype,vis_tidal,b,b_err,M_R90,M_R90_err,fit_function,A_g,A_r,logH2,R90_kpc
int64,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,float64,float64,bytes53,int64,bytes32,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,bytes32,int64,bytes19,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,bytes6,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001,12701,10001-12701,1-48157,v3_1_1,v3_1_1,v1_9_1,v5_5_36,v2_9,APOGEE-2&MaNGA,MaNGA dither,133.371090612,57.5984251446,159.27921887,38.8381829181,133.37109,57.598425,0.0545561,9,8100.81,0,20.0835,41.292,ma079,22,3,9114,134.41357,57.604214,1.10715,1.15321,1.21404,1.78219,1.85994,2.12685,0.898721,0.929654,0.937466,57372,57373,57373,2.69906,2.64546,2.59645,2.54039,2080,0,0,1,plateTargets-1.par,6013,J085329.03+573554.9,127,127,0,0.0392074,0.0193562,0.0264775,0.0386959,0.0429176,0.0193562,0.0264775,0.748562,-999.0,0.665321,0.705378,1.08789,1.19925,-999.0,0.66562,0.705713,203,1350,6,v1_0_1,51226,-999,0.0392074,0.0395926,-16.2982 .. -19.5112,-16.2321 .. -19.5522,275.541 .. 452.944,3068010000.0,2685340000.0,0.417013,69.2463,0.448783 .. 0.0800664,5.16633,3.63987,12.7947 .. 413.049,3.26768 .. 0.000422625,13.8306 .. 425.941,3.03694 .. 0.00496838,0.335333,0.741845,68.2584,5.29184,14.8124 .. 412.19,2.50888 .. 0.0150605,1.560186343059074,9.852487564086914,2.946989472970758,0.09848277794062997,0.8525269160441389,0.004661246847300842,38.7739389484797,0.022936739976051144,37.440940071078124,0.016138461970098106,61.88507825188626,0.07338042747869138,183.15428191832711,2.8318738759310786,2.2299464843830368,0.006099342053787376,10.529260284502032,4.181226771589277,0.07058094055808059,6.435441132989916,11.185428134112588,10.663096428880776,1.5,300.9007693953248,15.203381761428345,0.423009187

In [8]:
H_0 = 100      # Hubble's Constant in units of h km/s/Mpc
c = 299792.458 # Speed of light in units of km/s

In [9]:
master_table['v_3p5'] = np.ones(len(master_table))*np.nan
master_table['v_3p5_err'] = np.ones(len(master_table))*np.nan

In [14]:
for i in range (0,len(master_table)):
    
    if i % 50 == 0:
        print(i)
        
    if master_table['M'][i] > 0:
    
        gal_ID = master_table['plateifu'][i]
        #i_params = np.where(tab['plateifu'][i] == tab2['plateifu'])[0][0]
        z = master_table['z'][i]
    
        params = [master_table['v_max'][i], master_table['r_turn'][i], master_table['alpha'][i]]
        #params = [tab2['v_max'][i_params], tab2['r_turn'][i_params], tab2['alpha'][i_params]]
        #r90 = tab2['nsa_elpetro_th90'][i_params]
        R90 = master_table['R90_kpc'][i]

        ############################################################################
        # Convert r90 from arcsec to kpc
        #---------------------------------------------------------------------------
        #dist_to_galaxy_Mpc = c*z/H_0
        #dist_to_galaxy_kpc = dist_to_galaxy_Mpc*1000

        #r90_kpc = dist_to_galaxy_kpc*np.tan(r90*(1./60)*(1./60)*(np.pi/180))
        ############################################################################

        ############################################################################
        # Calculate velocity at given radius
        #---------------------------------------------------------------------------
        
        try:
            hess = np.load(HESS_FOLDER + gal_ID + '_Hessian.npy')
        except:
            continue

        N_samples = 10000

        v_samples = np.zeros(N_samples, dtype=float)

        v = rot_fit_BB(3.5*R90, params) # params is the set of best-fit values for the model
        master_table['v_3p5'][i] = v

        try:

            param_samples = np.random.multivariate_normal(mean=params, cov=0.5*hess[-3:,-3:], size=N_samples)

            for j in range(N_samples):

                if np.all(param_samples[j] > 0):
                    v_samples[j] = rot_fit_BB(3.5*R90, param_samples[j])

            v_err = np.nanstd(v_samples)
            master_table['v_3p5_err'][i] = v_err

        except np.linalg.LinAlgError:
            v_err = np.nan
            master_table['v_3p5_err'][i] = v_err

        ############################################################################

0


/Users/nityaravi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: covariance is not positive-semidefinite.


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150
1

In [18]:
master_table.write(OUTPUT_FOLDER+'H_alpha_HIvel_BB_extinction_H2_R90_v3p5.fits', format='fits', overwrite=True)